In [62]:
import pandas as pd
import os
import glob
from conn.apis import *
from pprint import *
# from langchain import *

In [63]:
directory = "ofertas"
dataframes = []

for file in glob.glob(os.path.join(directory, "*.xml")):
    # Ler o XML e converter para DataFrame
    df = pd.read_xml(file)
    df['name_file'] = file[8: len(file) -4]
    # Adicionar o DataFrame à lista
    dataframes.append(df)

df = pd.concat(dataframes, ignore_index=True)

print(len(df))
df.head()

6065


,sku,name,price,link,thumbnail,description,installment,name_file
0,DJ6258002,Tênis Nike Tanjun Next Nature Masculino,332.49,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/01429...,NaN,NaN,ofertas-nike
1,DN4181010,Plus Size - Top Nike Dri-FIT Indy Feminino,161.49,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/02319...,NaN,NaN,ofertas-nike
2,DV2992010,Boné Nike Dri-FIT AeroBill Legacy91 Unissex,151.99,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/02331...,NaN,NaN,ofertas-nike
3,CV4784010,Regata NikeCourt Victory Feminina,199.49,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/01084...,NaN,NaN,ofertas-nike
4,CZ0294602,Tênis Nike Court Legacy Canvas Feminino,313.49,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/01142...,NaN,NaN,ofertas-nike


In [64]:
df['firstName'] = df['name'].str.split().str[0]
df.head()

,sku,name,price,link,thumbnail,description,installment,name_file,firstName
0,DJ6258002,Tênis Nike Tanjun Next Nature Masculino,332.49,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/01429...,NaN,NaN,ofertas-nike,Tênis
1,DN4181010,Plus Size - Top Nike Dri-FIT Indy Feminino,161.49,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/02319...,NaN,NaN,ofertas-nike,Plus
2,DV2992010,Boné Nike Dri-FIT AeroBill Legacy91 Unissex,151.99,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/02331...,NaN,NaN,ofertas-nike,Boné
3,CV4784010,Regata NikeCourt Victory Feminina,199.49,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/01084...,NaN,NaN,ofertas-nike,Regata
4,CZ0294602,Tênis Nike Court Legacy Canvas Feminino,313.49,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/01142...,NaN,NaN,ofertas-nike,Tênis


In [65]:
grouped_df = df.groupby('firstName').agg(
    count=('firstName', 'size'),
    total_price=('price', 'sum')
).sort_values(by='count', ascending=False).reset_index()
print(len(grouped_df))

df_filtrado = grouped_df[grouped_df['count'] > 20]

print(len(df_filtrado))

199
30


In [95]:
df_filtrado

,firstName,count,total_price
0,Tênis,1396,945074.54
1,Camiseta,906,166345.44
2,Shorts,579,122002.71
3,Chuteira,336,243329.64
4,Torneira,245,50246.50
5,Calça,203,65690.47
6,Camisa,187,65557.63
7,Regata,174,46614.76
8,Jaqueta,156,72217.44
9,Blusão,155,49493.45


In [97]:
df_merged = pd.merge(df_filtrado, df, on='firstName', how='inner')

print(len(df_merged))
df_merged.head()
df_merged['sequence'] = df_merged.groupby('firstName').cumcount() + 1
df_merged = df_merged[df_merged['sequence'] <= 20]
df_merged = df_merged[df_merged['firstName'] != 'Torneira']
df_merged = df_merged[df_merged['firstName'] != 'Kit']
df_merged = df_merged[df_merged['firstName'] != 'Chuveiro']
df_merged = df_merged[df_merged['firstName'] != 'Ducha']
df_merged = df_merged[df_merged['firstName'] != 'Ventilador']

df_merged = df_merged.drop(columns=['sequence'])
df_merged = df_merged.dropna(subset=['name'])
df_merged = df_merged[df_merged['firstName'] != ""].reset_index()
print(len(df_merged))

df_merged.head()


5472
500


,index,firstName,count,total_price,sku,name,price,link,thumbnail,description,installment,name_file
0,0,Tênis,1396,945074.54,DJ6258002,Tênis Nike Tanjun Next Nature Masculino,332.49,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/01429...,NaN,NaN,ofertas-nike
1,1,Tênis,1396,945074.54,CZ0294602,Tênis Nike Court Legacy Canvas Feminino,313.49,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/01142...,NaN,NaN,ofertas-nike
2,2,Tênis,1396,945074.54,DA2778003,Tênis Nike Star Runner 3 Infantil,303.99,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/01217...,NaN,NaN,ofertas-nike
3,3,Tênis,1396,945074.54,DH1042091,Tênis NikeCourt Zoom Lite 3 Feminino,417.99,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/01364...,NaN,NaN,ofertas-nike
4,4,Tênis,1396,945074.54,DH3161101,Tênis Nike Court Legacy Next Nature Feminino,360.99,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,https://imgnike-a.akamaihd.net/1300x1300/01371...,NaN,NaN,ofertas-nike


In [98]:
#Teste de inserção de 2 valors
dfFinal = df_merged
dfFinal.rename(columns={'sku': 'idProduto'}, inplace=True)
dfFinal.rename(columns={'name_file': 'loja'}, inplace=True)

json_result = dfFinal[1:3].to_json(orient='records')

retorno1 = put_products(json_result)
pprint(json_result)
pprint(retorno1)

('[{"index":1,"firstName":"T\\u00eanis","count":1396,"total_price":945074.54,"idProduto":"CZ0294602","name":"T\\u00eanis '
 'Nike Court Legacy Canvas '
 'Feminino","price":313.49,"link":"https:\\/\\/redir.lomadee.com\\/v2\\/direct\\/aHR0cHM6Ly93d3cubmlrZS5jb20uYnIvdGVuaXMtd21ucy1uaWtlLWNvdXJ0LWxlZ2FjeS1jbnZzLTAxMTQyMi5odG1sP2Nvcj1NVQ==\\/LOMADEESOURCEID\\/5798","thumbnail":"https:\\/\\/imgnike-a.akamaihd.net\\/1300x1300\\/011422MUA9.jpg","description":null,"installment":null,"loja":"ofertas-nike"},{"index":2,"firstName":"T\\u00eanis","count":1396,"total_price":945074.54,"idProduto":"DA2778003","name":"T\\u00eanis '
 'Nike Star Runner 3 '
 'Infantil","price":303.99,"link":"https:\\/\\/redir.lomadee.com\\/v2\\/direct\\/aHR0cHM6Ly93d3cubmlrZS5jb20uYnIvdGVuaXMtbmlrZS1zdGFyLXJ1bm5lci0zLWluZmFudGlsLTAxMjE3NS5odG1sP2Nvcj1JRQ==\\/LOMADEESOURCEID\\/5798","thumbnail":"https:\\/\\/imgnike-a.akamaihd.net\\/1300x1300\\/012175IEA8.jpg","description":null,"installment":null,"loja":"ofertas-nike"}]')


In [99]:
chunk_size = 30

for start in range(0, len(dfFinal), chunk_size):
    end = start + chunk_size
    data_chunk = dfFinal.iloc[start:end]
    data_chunk_json = data_chunk.to_json(orient='records')
    retorno = put_products(data_chunk_json)
    print(retorno)
    print(start)

[]
0
[]
10
[]
20
[]
30
[]
40
[]
50
[]
60
[]
70
[]
80
[]
90
[]
100
[]
110
[]
120
[]
130
[]
140
[]
150
[]
160
[]
170
[]
180
[]
190
[]
200
[]
210
[]
220
[]
230
[]
240
[]
250
[]
260
[]
270
[]
280
[]
290
[]
300
[]
310
[]
320
[]
330
[]
340
[]
350
[]
360
[]
370
Erro na chamada de API (status code 502).
380
Erro na chamada de API (status code 502).
390
[]
400
[]
410
[]
420
[]
430
[]
440
[]
450
[]
460
[]
470
[]
480
[]
490


In [100]:
retorno = get_all_products()

In [105]:

getDf = pd.DataFrame(retorno)
getDf

,total_price,index,count,installment,idProduto,link,loja,thumbnail,price,description,name,firstName
0,65557.63,3665.0,187.0,None,DJ7780078,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,ofertas-nike,https://imgnike-a.akamaihd.net/1300x1300/02261...,322.99,None,Camisa Nike PSG II 2022/23 Torcedora Pro Feminina,Camisa
1,21829.81,4475.0,119.0,None,FB4095457,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,ofertas-nike,https://imgnike-a.akamaihd.net/1300x1300/02769...,180.49,None,Top Jordan Indy Feminino,Top
2,31690.80,4345.0,120.0,None,DQ8917536,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,ofertas-nike,https://imgnike-a.akamaihd.net/1300x1300/02523...,227.99,None,Legging Nike Yoga Dri-FIT Infantil,Legging
3,13641.60,5216.0,40.0,None,943826501,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,ofertas-nike,https://imgnike-a.akamaihd.net/1300x1300/00400...,294.49,None,Sandália Nike Sunray Protect 2 Infantil,Sandália
4,166345.44,1406.0,906.0,None,DD4921609,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,ofertas-nike,https://imgnike-a.akamaihd.net/1300x1300/01332...,199.49,None,Camiseta Nike Dri-FIT One Luxe Feminina,Camiseta
...,...,...,...,...,...,...,...,...,...,...,...,...
476,46614.76,3862.0,174.0,None,DV9189010,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,ofertas-nike,https://imgnike-a.akamaihd.net/1300x1300/02591...,180.49,None,Regata Nike Yoga Dri-FIT Feminina,Regata
477,21829.81,4460.0,119.0,None,DV9837363,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,ofertas-nike,https://imgnike-a.akamaihd.net/1300x1300/02487...,199.49,None,Top Nike Dri-FIT Indy Feminino,Top
478,3220.29,5452.0,21.0,None,FB3042838,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,ofertas-nike,https://imgnike-a.akamaihd.net/1300x1300/02742...,142.49,None,Pochete Nike Heritage Unissex,Pochete
479,17127.83,4864.0,67.0,None,CN9675400,https://redir.lomadee.com/v2/direct/aHR0cHM6Ly...,ofertas-nike,https://imgnike-a.akamaihd.net/1300x1300/00935...,189.99,None,Chinelo Nike Victori One Masculino,Chinelo


In [106]:
grouped_getDf = getDf.groupby('firstName').agg(
    count=('firstName', 'size'),
    total_price=('price', 'sum')
).sort_values(by='count', ascending=False).reset_index()
grouped_getDf

,firstName,count,total_price
0,Agasalho,20,7428.80
1,Air,20,20652.80
2,Blusão,20,5671.30
3,Bola,20,3562.30
4,Bolsa,20,5072.80
5,Boné,20,2830.80
6,Calça,20,6136.80
7,Camisa,20,6573.80
8,Camiseta,20,3562.30
9,Chinelo,20,4493.30
